In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [69]:
data = pd.read_csv('/kaggle/input/heart-disease-prediction/public_datasets/Heart_disease_cleveland_new.csv')

In [70]:
CTGANSynthesizer_data = pd.read_csv('/kaggle/input/heart-disease-prediction/synthetic_datasets/CTGANSynthesizer_dataset.csv')

In [71]:
X = data.drop('target', axis=1).astype(float)
y = data['target'].astype(int)
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63.0,1.0,0.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,2.0,0.0,2.0
1,67.0,1.0,3.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,1.0,3.0,1.0
2,67.0,1.0,3.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,1.0,2.0,3.0
3,37.0,1.0,2.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,2.0,0.0,1.0
4,41.0,0.0,1.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,0.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,1.0,0.0,3.0
299,68.0,1.0,3.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,1.0,2.0,3.0
300,57.0,1.0,3.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,1.0,1.0,3.0
301,57.0,0.0,1.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,1.0,1.0,1.0


In [72]:
X_CTGANSynthesizer = CTGANSynthesizer_data.drop('target', axis=1).astype(float)

In [73]:
y_CTGANSynthesizer = CTGANSynthesizer_data['target'].astype(int)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = pd.concat([X_train, X_CTGANSynthesizer], ignore_index=True)
y_train = pd.concat([y_train, y_CTGANSynthesizer], ignore_index=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [75]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [76]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.int64)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.int64)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.int64)

In [77]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [78]:
class HeartDiseaseClassifier(nn.Module):
    def __init__(self):
        super(HeartDiseaseClassifier, self).__init__()
        self.fc1 = nn.Linear(13, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Output size is 2 for binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [79]:
model = HeartDiseaseClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [80]:
best_val_loss = float('inf')
best_model_path = '/kaggle/working/CTGANSynthesizer_best_model.pt'

In [81]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
        val_loss /= len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        print("Saving best model...")

Epoch [1/50], Validation Loss: 0.6792
Saving best model...
Epoch [2/50], Validation Loss: 0.6756
Saving best model...
Epoch [3/50], Validation Loss: 0.6693
Saving best model...
Epoch [4/50], Validation Loss: 0.6689
Saving best model...
Epoch [5/50], Validation Loss: 0.6640
Saving best model...
Epoch [6/50], Validation Loss: 0.6662
Epoch [7/50], Validation Loss: 0.6681
Epoch [8/50], Validation Loss: 0.6693
Epoch [9/50], Validation Loss: 0.6738
Epoch [10/50], Validation Loss: 0.6729
Epoch [11/50], Validation Loss: 0.6793
Epoch [12/50], Validation Loss: 0.6783
Epoch [13/50], Validation Loss: 0.6834
Epoch [14/50], Validation Loss: 0.6835
Epoch [15/50], Validation Loss: 0.6894
Epoch [16/50], Validation Loss: 0.6905
Epoch [17/50], Validation Loss: 0.6977
Epoch [18/50], Validation Loss: 0.6981
Epoch [19/50], Validation Loss: 0.7006
Epoch [20/50], Validation Loss: 0.7041
Epoch [21/50], Validation Loss: 0.7125
Epoch [22/50], Validation Loss: 0.7157
Epoch [23/50], Validation Loss: 0.7185
Epoch [

In [82]:
model.load_state_dict(torch.load(best_model_path))

<All keys matched successfully>

In [83]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {(100 * correct / total):.2f}%')

Accuracy on test set: 83.61%
